In [16]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import sys
import time

### 读取数据

In [17]:
def load_dataset(dataSet, batch_size=64):
    if sys.platform.startswith('win'):
        num_workers = 0  # 0表示不用额外的进程来加速读取数据
    else:
        num_workers = 4
        print('linux')
    dataset_iter = torch.utils.data.DataLoader(dataSet, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    return dataset_iter

### 实现softmax运算

In [21]:
def softmax(X):
    x_exp = X.exp()
    sum = x_exp.sum(dim=1, keepdim=True)
    return x_exp / sum

### 定义模型

In [60]:
def model(data, w, b):
    return softmax(torch.mm(data.view(-1, 784), w) + b)

### 定义损失函数

In [157]:
def loss(lable, y):
    # print(lable.view(-1, 1))
    return - torch.log(y.gather(1, lable.view(-1, 1)))

### 定义优化算法

In [158]:
def sgd(params, lr, batch_size): 
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

### 计算准确率

In [159]:
def accuracy(lable, y):
    return (y.argmax(dim=1) == y).float.mean().item()

### 评价准确率

In [160]:
def evaluate_accuracy(data_iter, parmas):
    acc_sum, n = 0.0, 0
    w, b = params
    for X, y in data_iter:
        acc_sum += (model(X, w, b).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

### 训练模型

In [161]:
def training_loop(net, loss, n_epochs, params, train_dataset, test_dataset, batch_size=64, lr=0.001,):
    train_iter = load_dataset(train_dataset, batch_size)
    test_iter = load_dataset(test_dataset, batch_size)
    for epoch in range(n_epochs+1):
        train_loss_sum, train_acc_sum, n = 0.0, 0.0, 0
        for x, lable in train_iter:
            n += lable.shape[0]
            w, b = params
            y = net(x, w, b)
            l = loss(lable, y).sum()
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()  # 梯度清零
            l.backward()
            sgd(params, lr, batch_size)
            train_loss_sum += l.item()
            train_acc_sum += (y.argmax(dim=1) == lable).sum().item()
        test_acc = evaluate_accuracy(test_iter, params)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f,  train acc2_: %.3f,'
              % (epoch + 1, train_loss_sum / n, train_acc_sum / n, test_acc, evaluate_accuracy(train)))

In [162]:
if __name__ == '__main__':
    savePath = './Fashion-MNIST/'
    mnist_train = torchvision.datasets.FashionMNIST(root=savePath, train=True, download=True, transform=transforms.ToTensor())
    mnist_test = torchvision.datasets.FashionMNIST(root=savePath, train=False, download=True, transform=transforms.ToTensor())
    
    # 初始化模型参数
    num_inputs = 784
    num_outputs = 10
    w = torch.tensor(np.random.normal(0, 0.1, (num_inputs, num_outputs)), dtype=torch.float)
    b = torch.zeros(num_outputs, dtype=torch.float)
    w.requires_grad_(requires_grad=True)
    b.requires_grad_(requires_grad=True) 
    print(b.shape)
    
    n_epochs=1
    params = [w, b]
    lr = 0.001
    training_loop(net=model, loss=loss, n_epochs=n_epochs, batch_size=64, params=params, 
                  lr=0.001,train_dataset=mnist_train, test_dataset=mnist_test)

torch.Size([10])
linux
linux
tensor([[1.1479],
        [3.4923],
        [2.7604],
        [6.2296],
        [1.1171],
        [1.1789],
        [1.8271],
        [2.4174],
        [3.4046],
        [4.2936],
        [2.6558],
        [0.6352],
        [1.8937],
        [2.0271],
        [3.7170],
        [2.7519],
        [2.7019],
        [3.8432],
        [2.6398],
        [2.1262],
        [2.9771],
        [3.1874],
        [1.5085],
        [1.7312],
        [1.8672],
        [2.4381],
        [1.8760],
        [2.8743],
        [2.9577],
        [1.8012],
        [6.5234],
        [1.1647],
        [2.6920],
        [2.1385],
        [1.3151],
        [3.2109],
        [1.6675],
        [1.7330],
        [2.9513],
        [2.0621],
        [2.2862],
        [4.1587],
        [0.8628],
        [3.9929],
        [4.0776],
        [3.1462],
        [3.2456],
        [2.7446],
        [3.4926],
        [2.9272],
        [3.6234],
        [3.4317],
        [2.4454],
        [5.4330],

tensor([[3.0342],
        [1.6417],
        [2.1953],
        [0.7410],
        [1.4490],
        [3.0819],
        [2.0468],
        [3.2867],
        [1.0993],
        [3.4004],
        [2.7134],
        [1.7543],
        [3.3717],
        [2.7626],
        [1.4492],
        [2.8181],
        [2.7785],
        [1.6664],
        [2.5346],
        [3.1903],
        [5.4821],
        [1.9740],
        [3.1305],
        [3.3483],
        [3.1165],
        [4.4653],
        [1.9831],
        [2.6234],
        [1.7813],
        [1.6540],
        [2.8464],
        [3.2928],
        [1.1073],
        [1.9879],
        [1.2221],
        [3.8581],
        [3.2880],
        [1.8977],
        [1.4804],
        [3.1760],
        [2.2849],
        [3.3771],
        [3.5995],
        [1.7475],
        [2.2551],
        [3.1927],
        [4.4238],
        [3.3768],
        [2.9157],
        [2.6953],
        [2.9575],
        [2.6044],
        [2.5257],
        [2.4207],
        [1.2160],
        [2

        [2.8066]], grad_fn=<NegBackward>)
tensor([[1.5168],
        [1.7977],
        [1.8321],
        [3.0632],
        [1.5687],
        [1.9352],
        [2.5429],
        [1.4764],
        [1.1411],
        [2.2393],
        [2.1313],
        [1.8840],
        [2.6070],
        [3.7609],
        [1.8225],
        [2.1222],
        [1.5682],
        [4.6063],
        [2.2073],
        [5.6506],
        [1.4268],
        [2.3812],
        [2.2273],
        [2.7916],
        [1.8740],
        [3.7645],
        [2.6804],
        [2.7117],
        [1.5096],
        [2.5577],
        [2.6924],
        [1.4737],
        [3.6095],
        [3.6387],
        [1.8228],
        [2.3452],
        [1.3874],
        [2.4762],
        [2.0622],
        [2.2925],
        [3.0786],
        [4.2148],
        [1.2928],
        [1.7084],
        [1.9234],
        [2.5935],
        [1.8001],
        [2.0712],
        [2.1050],
        [2.3123],
        [2.8360],
        [1.3492],
        [1.3848],
    

tensor([[1.6202],
        [1.9053],
        [2.2232],
        [2.3578],
        [0.8646],
        [2.0287],
        [2.7666],
        [2.2723],
        [0.8234],
        [1.8441],
        [2.6317],
        [1.6862],
        [1.5535],
        [5.3423],
        [2.1072],
        [1.8489],
        [1.5147],
        [2.2276],
        [1.3682],
        [2.0017],
        [2.9498],
        [2.2829],
        [1.1600],
        [1.8060],
        [1.7448],
        [4.0878],
        [1.8762],
        [1.5825],
        [2.0183],
        [2.1089],
        [2.5636],
        [0.8500],
        [1.8046],
        [2.0561],
        [1.9530],
        [1.9125],
        [5.5325],
        [2.5136],
        [2.5789],
        [1.4197],
        [3.6517],
        [2.3041],
        [1.0948],
        [2.3509],
        [3.7168],
        [1.9660],
        [5.5735],
        [3.4703],
        [3.4499],
        [0.5923],
        [2.3819],
        [2.4613],
        [2.1013],
        [1.7465],
        [2.4174],
        [2

tensor([[2.4490],
        [2.4179],
        [2.1136],
        [2.5819],
        [2.5607],
        [2.1479],
        [1.3591],
        [1.9953],
        [5.2454],
        [3.6887],
        [2.2328],
        [2.7038],
        [1.1648],
        [4.1436],
        [0.7031],
        [1.4013],
        [5.2502],
        [4.1681],
        [2.6332],
        [2.4314],
        [1.9930],
        [1.2696],
        [2.4102],
        [2.3673],
        [5.8442],
        [2.2623],
        [4.9008],
        [3.2156],
        [2.4762],
        [2.6244],
        [3.0319],
        [1.7522],
        [2.5578],
        [4.6933],
        [3.4086],
        [1.9488],
        [2.0807],
        [1.9592],
        [1.7171],
        [1.8658],
        [2.3431],
        [2.0370],
        [2.1312],
        [2.3271],
        [5.5294],
        [1.4072],
        [1.9612],
        [2.2949],
        [2.8168],
        [2.3333],
        [0.9816],
        [2.6787],
        [0.7786],
        [1.5961],
        [1.4881],
        [2

...

tensor([[1.0394],
        [4.9745],
        [2.8548],
        [1.7101],
        [4.2656],
        [0.6401],
        [1.9007],
        [2.0948],
        [0.7516],
        [0.4964],
        [1.1508],
        [2.0721],
        [1.3944],
        [0.9655],
        [2.3555],
        [2.5068],
        [1.4968],
        [0.6715],
        [1.1885],
        [2.8355],
        [0.7976],
        [2.0918],
        [0.9381],
        [1.1603],
        [2.1568],
        [0.6890],
        [0.8042],
        [0.8335],
        [0.4991],
        [0.9036],
        [1.7489],
        [0.6920],
        [1.2190],
        [1.7699],
        [0.8144],
        [0.2957],
        [0.9234],
        [2.4869],
        [0.9091],
        [1.9023],
        [1.6183],
        [1.4235],
        [1.8453],
        [1.1274],
        [0.9071],
        [1.0318],
        [1.3392],
        [1.6470],
        [1.6313],
        [1.7468],
        [1.5049],
        [2.1328],
        [2.4508],
        [0.7166],
        [1.7368],
        [2

tensor([[0.6566],
        [1.0762],
        [1.0372],
        [2.2138],
        [1.2630],
        [1.1498],
        [1.3547],
        [0.9729],
        [0.6039],
        [1.1156],
        [1.3780],
        [0.2961],
        [1.1517],
        [0.9914],
        [2.3771],
        [1.7913],
        [0.9359],
        [1.3477],
        [1.2469],
        [1.1428],
        [0.6989],
        [2.0816],
        [1.5323],
        [1.7082],
        [2.4822],
        [0.1407],
        [1.6067],
        [2.1147],
        [2.9163],
        [0.6203],
        [1.7889],
        [1.7173],
        [0.5692],
        [0.9883],
        [2.6056],
        [2.1825],
        [0.6346],
        [1.3316],
        [0.6536],
        [2.0896],
        [2.6025],
        [0.9319],
        [0.9296],
        [1.2803],
        [2.3658],
        [1.9179],
        [3.6890],
        [0.6955],
        [1.1748],
        [1.6733],
        [2.2136],
        [0.8590],
        [1.6379],
        [1.2340],
        [0.9529],
        [2

tensor([[0.5153],
        [1.4885],
        [1.0964],
        [0.3392],
        [1.9389],
        [1.1008],
        [2.0519],
        [1.2686],
        [2.3768],
        [2.4337],
        [1.0038],
        [1.2334],
        [1.1049],
        [0.9666],
        [2.5482],
        [1.0343],
        [2.2349],
        [1.3054],
        [0.7456],
        [0.5003],
        [1.6483],
        [0.8366],
        [0.8436],
        [1.5330],
        [0.8549],
        [2.1685],
        [0.7198],
        [2.0037],
        [0.8822],
        [2.6561],
        [1.3395],
        [0.5589],
        [0.9797],
        [1.5580],
        [2.3436],
        [0.2396],
        [1.4477],
        [0.9078],
        [2.1515],
        [0.8480],
        [1.9849],
        [0.7633],
        [1.1335],
        [1.7535],
        [1.7062],
        [0.4685],
        [2.3026],
        [1.3136],
        [0.6324],
        [0.7040],
        [0.9477],
        [0.7374],
        [1.7076],
        [0.7285],
        [2.3377],
        [0

tensor([[2.5079],
        [1.2955],
        [1.7901],
        [1.0159],
        [1.5072],
        [2.2011],
        [0.7095],
        [1.1092],
        [0.8704],
        [1.5100],
        [2.0662],
        [1.8966],
        [1.1177],
        [0.8658],
        [0.8240],
        [1.5529],
        [0.8602],
        [0.8991],
        [1.0804],
        [2.2493],
        [2.2301],
        [0.5422],
        [2.2830],
        [1.1410],
        [1.1868],
        [1.3016],
        [0.8084],
        [1.2359],
        [1.8434],
        [2.0135],
        [1.0722],
        [0.8895],
        [0.7657],
        [2.0812],
        [2.2700],
        [1.4675],
        [1.5557],
        [2.1733],
        [0.7798],
        [0.6033],
        [0.9771],
        [0.7777],
        [2.2571],
        [0.4760],
        [2.1538],
        [2.0748],
        [1.0322],
        [0.8653],
        [2.3839],
        [1.4293],
        [1.0794],
        [1.1204],
        [0.6074],
        [1.6657],
        [1.7043],
        [1

NameError: name 'train' is not defined